Linear Data Lab 10

Original lab written by: Emily J. King

Goals: Properly encode systems of linear equations into a computer and solve it using least squares.  Calculate fundamental spaces of matrices, recognizing when returned value is understood as an affine space and when it is an actual vector. Eigenvalue teaser.

Additional file needed: Linear_Data_Chapter_10_Lab.pdf

In [ ]:
import numpy as np
from numpy.linalg import det
from numpy.linalg import inv
from numpy.linalg import matrix_rank as rank
from scipy.linalg import null_space as null
from numpy.linalg import lstsq as lsqr

Section 1: Traffic problem

Read about the system of linear equations arising from a traffic problem on Linear_Data_Chapter_10_Lab.pdf.

Let's write the traffic problem as a matrix equation Ax=b.

In [ ]:
A=np.array([[1, -1, 0, 0],[0, 1, -1, 0],[0, 0, 1, -1],[-1, 0, 0, 1]])
b=np.array([160,-40,210,-330])

We'd like to find an x such that Ax=b.  Since A is square, let's just try computing x=A^(-1)b.

In [ ]:
inv(A)@b

We got an error because A is not an invertible matrix.  So, we'll need to find another way to try to solve the system.

Even though A is not invertible, this system is actually nice enough that one could solve for all of the solutions by hand (using so-called row operations).  However, often systems arising in real life involve measurements, and do not have an exact solution.  Also, often, one wants to make additional assumptions about your desirend solution (e.g., it has some special structure), and thus would solve, e.g., some so-called regularized optimization problem.

In any case, in this class we will be using least squares to solve both explicit least squares problems and systems of linear equations.

The following command will return a solution to the least squares problem minimizing ||Ax-b||.  Note that any solution (if it exists) to Ax=b must be a solution to least squares since then the Euclidean distance between Ax and b would be 0.  I.e., min_x ||Ax-b|| = 0 if and only if Ax=b has a solution.

In [ ]:
x1=lsqr(A,b,rcond=None)[0]
x1

Let's test to see if this is indeed a solution to the original equation.

In [ ]:
np.allclose(A@x1,b)

You should have gotten True since this system has a solution.  Most likely, however, the given solution x1 isn't feasible for the orginal word problem (i.e., has negatve traffic flow) even though Ax1 = b.  

Since Ax1=b, this also means that b is in the image of A.  (Also known as the column space of A.)

But is x1 the only solution?  Since A is square but not invertible, rank-nullity tells us that it must have a non-trivial nullspace  Let's check.

In [ ]:
rank(A)

So, by rank-nullity, A has a one-dimensional null space.

In [ ]:
N=null(A)
N

The null command returns an orthonormal basis, listed as columns of a matrix, for the nullspace/kernel of the matrix A.  Since null(A) returns a basis for the nullspace, this means that every vector that goes to zero under multiplication by this matrx is a linear combination of the vectors returned by null(A). Also note that since  null(A) returns a basis (in the form of a matrix) for the nullspace, we have confirmed that the nullspace of this particular matrix has dimension 1. 

To compute with the basis as a vector, we are going to take the only column.

In [ ]:
A@N[:,0]

We get, as expected, the zero vector up to machine precision when multiplying the single basis vector for this null space by A.  We can use this to generate more solutions to Ax=b.

In [ ]:
np.allclose(A@(x1+N[:,0]),b)

So adding the basis vector for the null space to our original solution x1 gives us another solution.  Actually adding anything in the null space, i.e., the span of the basis of the nullspace, yields another solution. Next, we're adding a random scalar multiple.

In [ ]:
np.allclose(A@(x1+np.random.normal()*N[:,0]),b)

Section 2: More on fundamental spaces

Let's consider a 5x6 matrix C.

In [ ]:
C=np.array([[3, 10, -2, 2, 4, -6],[4, 8, 0, 0, 0, 0],[-3, 1, 4, 7, 3, 1],[4, -3, 4, 1, -3, 7],[1, 3, -3, -4, -1, -2]])
C

Let's compute a basis for the nullspace.

In [ ]:
NC=null(C)
NC

So, any random linear combination of the two columns of N will be in the nullspace of C.

In [ ]:
C@(np.random.normal()*NC[:,0]+np.random.normal()*NC[:,1])

To generate a random element of the image of C, we just need to multiply a random vector of the correct size so that the matrix multiplication is defined.

In [ ]:
C@np.random.rand(6)

Let's test to see if a given vector is in the image of C.

In [ ]:
z=np.array([30,-65,46,50,108])
rank(C)==rank(np.column_stack((C,z)))

So, the dimension of the span of the columns of C is not the same as the dimension of the span of the columns of C and z, meaning that z is not in the span, i.e., not in the image of C. 

However, we can still solve the least squares problem min_x ||Cx - z||.

In [ ]:
xCz=lsqr(C,z,rcond=None)[0]
np.allclose(C@xCz,z)

So, xCz is a solution to the least squares problem min_x ||Cx - z||, but there is no solution to Cx=z.

Section 3: Eigenvalue Teaser

Input the matrices from Linear_Data_Chapter_10_Lab.pdf.  

Note that np.cos(np.radians()) and np.sin(np.radians()) are the Numpy commands for cosine and sine with degrees as the unit of inputs.  Also note that np.diag(vector with d entries) yields a dxd diagonal matrix with diagonal entries the entries of the vector.

In [ ]:
D1 = np.diag(np.array([0.5, 3]))
R = np.array([[np.cos(np.radians(30)), -np.sin(np.radians(30))], [np.sin(np.radians(30)), np.cos(np.radians(30))]])

Define M to be R*D1*R^(-1)

In [ ]:
M=R@D1@inv(R)

In [ ]:
det(M)

Since det(M) is non-zero, M is invertible. Compare M^(-1) to R*D1^(-1)*R^(-1).

In [ ]:
np.allclose(inv(M),R@inv(D1)@inv(R))

Recall properties of matrix inverses that explain the result above. Discuss.

Let r1 and r2 be the first and second columns of R.

In [ ]:
r1=R[:,0]
r2=R[:,1]

Multiply M(r1)

In [ ]:
M@r1

We will compare M(r1) to r1 by dividing COMPONENT-WISE.  This is not a linear algebra operation from class, but it will help us see the relationship between the two vectors. 

In [ ]:
M@r1/r1

The above calculation tells us that Mr1 = 0.5r1.  Now let's test M(r2):

In [ ]:
M@r2/r2

It has the property that Mr2=3r2.

Exercises

1a. Define T to be a random 3x6 matrix and c a random 3x1 vector. 

b. Test if there is a solution to Tx=c.  If there is a solution to Tx=c, give an additional solution to the problem.

2a. Let B be a random 5x5 matrix and D a random 5x5 diagonal matrix.

b. Compute BDB^(-1) times the first column of B. 

c. How does the output of d compare to the first column of B before being acted on?